In [9]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [10]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [11]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-21
DEST_DIR: ..\data

source_path: ..\data\2025-04-21_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-21_df_perf_ratios.parquet


In [12]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [13]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
REXR   2025-04-21   32.11   32.22   31.08   31.69      31.69  2309512   32.110000   32.220000   31.080000
       2025-04-17   33.38   33.57   32.46   32.57      32.57  4910000   33.380000   33.570000   32.460000
       2025-04-16   32.60   33.19   32.35   32.70      32.70  3269900   32.600000   33.190000   32.350000
       2025-04-15   32.90   33.00   32.41   32.56      32.56  2875100   32.900000   33.000000   32.410000
       2025-04-14   33.01   33.23   32.35   32.82      32.82  2436100   33.010000   33.230000   32.350000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
KSPI   2024-04-26  117.00  121.72  116.14  120.12     116.51   233500  113.483766  118.061915  112.649612
       2024-04-25  113.54  118.34  111.16  116.32     112.82   244400  110.123649  114.779219  107.815261
       2024-04-24  122.19  124.08  115.88  116.20     112.70   280500  118.509578  120.342651  112.389639
       2024-04-23  117.50  123.17  114.95  122.99     119.29   347900  113.965160  119.464585  111.491873
       2024-04-22  119.10  122.56  114.28  115.18     111.71   496100  115.511903  118.867665  110.837114

[368500 rows x 9 columns]

In [14]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
REXR   2025-04-21      31.69
       2025-04-17      32.57
       2025-04-16      32.70
       2025-04-15      32.56
       2025-04-14      32.82
...                      ...
KSPI   2024-04-26     116.51
       2024-04-25     112.82
       2024-04-24     112.70
       2024-04-23     119.29
       2024-04-22     111.71

[368500 rows x 1 columns]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 368500 entries, ('REXR', Timestamp('2025-04-21 00:00:00')) to ('KSPI', Timestamp('2024-04-22 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  368500 non-null  float64
dtypes: float64(1)
memory usage: 4.3+ MB


In [16]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-21     193.16
       2025-04-17     196.98
       2025-04-16     194.27
       2025-04-15     202.14
       2025-04-14     202.52
...                      ...
IBDS   2024-04-26      22.53
       2024-04-25      22.52
       2024-04-24      22.54
       2024-04-23      22.56
       2024-04-22      22.52

[368750 rows x 1 columns]


In [17]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [19]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A    -13.871145  -12.340142  0.000000 -18.745054  -12.800841  0.000000   -1.119263    -1.695660   0.827912   -4.084646    -5.083582   0.514629   -3.490347    -4.346662   0.538093   -4.597900    -5.377312   0.433395    -1.638485     -2.159203    0.746554    -0.947078     -1.251221    0.843790
AA   -11.555912  -11.386837  0.000000  -7.338795   -7.653795  0.185497   -0.851168    -1.354425   0.859719   -3.989441    -5.054660   0.497382   -2.651998    -3.435725   0.631556   -2.718677    -3.410509   0.617469    -1.915549     -2.454478    0.709119    -0.654733     -0.898414    0.894898
AAL   -9.206931  -10.116292  0.098769  -5.974903   -6.991279  0.376914   -0.5437

In [20]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1474 entries, A to ZTS
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sharpe 3d     1474 non-null   float64
 1   Sortino 3d    1474 non-null   float64
 2   Omega 3d      1365 non-null   float64
 3   Sharpe 5d     1474 non-null   float64
 4   Sortino 5d    1474 non-null   float64
 5   Omega 5d      1453 non-null   float64
 6   Sharpe 10d    1474 non-null   float64
 7   Sortino 10d   1474 non-null   float64
 8   Omega 10d     1473 non-null   float64
 9   Sharpe 15d    1474 non-null   float64
 10  Sortino 15d   1474 non-null   float64
 11  Omega 15d     1474 non-null   float64
 12  Sharpe 30d    1474 non-null   float64
 13  Sortino 30d   1474 non-null   float64
 14  Omega 30d     1474 non-null   float64
 15  Sharpe 60d    1474 non-null   float64
 16  Sortino 60d   1474 non-null   float64
 17  Omega 60d     1474 non-null   float64
 18  Sharpe 120d   1474 non-null   floa

In [21]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            # NaN is from Omega ratio where daily loss series are all zero
            # +Inf is from Sharpe ratio where daily negative excess return series are all zero
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            # -Inf is from Sharpe ratio where daily excess return series are the same, therefore standard deviation is zero
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sortino 3d
  Inf: ['ACI', 'ADM', 'ALB', 'ALGN', 'AMX', 'ARKB', 'ARW', 'ASR', 'AVDV', 'AVEM', 'BABA', 'BBD', 'BBY', 'BCE', 'BCH', 'BF-A', 'BF-B', 'BHP', 'BIDU', 'BITB', 'BNTX', 'BPOP', 'BSAC', 'BSBR', 'BTI', 'CAG', 'CF', 'COF', 'CPB', 'DEO', 'DFAE', 'DFS', 'DG', 'DINO', 'DLTR', 'DPZ', 'EBR', 'EEMV', 'EFAV', 'ESGE', 'FBTC', 'FIS', 'FNDC', 'FTSM', 'FWONK', 'GBTC', 'GIS', 'GMAB', 'GPK', 'GSK', 'HDB', 'IBDR', 'IBIT', 'IBN', 'IFF', 'IMO', 'INDA', 'ITUB', 'JMST', 'K', 'KB', 'KHC', 'KR', 'KWEB', 'LEVI', 'LI', 'LULU', 'MCHI', 'MCHP', 'MDLZ', 'MGM', 'MSTR', 'MTCH', 'NEAR', 'NFLX', 'NGG', 'NKE', 'NTES', 'NXPI', 'ON', 'PDD', 'PTLC', 'PULS', 'QRVO', 'RBLX', 'RIO', 'ROKU', 'SCHO', 'SCZ', 'SKM', 'SMMT', 'SNY', 'STZ', 'SU', 'SWKS', 'TAK', 'TD', 'TGT', 'TKC', 'TU', 'U', 'ULTA', 'USFR', 'VALE', 'VGSH', 'VTRS', 'VWO', 'WYNN', 'YMM']
Column: Omega 3d
  NaN: ['ACI', 'ADM'

In [22]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'ACI'

#### Use Cell below to check the inf, -inf and NaN replacement

In [23]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       853.893685
Sortino 3d             inf
Omega 3d               NaN
Sharpe 5d         2.476010
Sortino 5d        3.852072
Omega 5d          1.485315
Sharpe 10d        2.215683
Sortino 10d       3.218091
Omega 10d         1.465911
Sharpe 15d        0.601949
Sortino 15d       0.822566
Omega 15d         1.104643
Sharpe 30d        1.173252
Sortino 30d       1.645921
Omega 30d         1.216251
Sharpe 60d        1.406869
Sortino 60d       2.023085
Omega 60d         1.275999
Sharpe 120d       1.453342
Sortino 120d      2.154772
Omega 120d        1.285105
Sharpe 250d       0.462784
Sortino 250d      0.671033
Omega 250d        1.084121
Name: ACI, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d        853.893685
Sortino 3d      2102.104872
Omega 3d         188.270386
Sharpe 5d          2.476010
Sortino 5d         3.852072
Omega 5d           1.485315
Sharpe 10d         2.215683
Sortino 10d        3.218091
Omega 10d          1.465911
Sharpe 15d         0.601949
Sortino 15d        0.822566
Omega 15d          1.104643
Sharpe 30d         1.173252
Sortino 30d        1.645921
Omega 30d          1.216251
Sharpe 60d         1.406869
Sortino 60d        2.023085
Omega 60d          1.275999
Sharpe 120d        1.453342
Sortino 120d       2.154772
Omega 120d         1.285105
Sharpe 250d        0.462784
Sortino 250d       0.671033
Omega 250d         1.084121
Name: ACI, dtype: float64

In [24]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-21_df_perf_ratios.parquet


In [28]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'TBIL']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====